## Load required libraries

In [1]:
import keras # imports keras and tensorflow as backend

import numpy as np
import pandas as pd

import os
import time

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard
import keras.backend as K

# from keras.utils import plot_model

Using TensorFlow backend.


## Save tensorboard logs with different runs.
### Run everytime when running model.

In [2]:
NAME = "DeepRecommender-Functional-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='./logs/{}'.format(NAME))

## Set Data Paths

In [3]:
DATA_DIR = './nf_data'

nf_3m_train = os.path.join(DATA_DIR, 'N3M_TRAIN', 'n3m.train.txt')
nf_3m_valid = os.path.join(DATA_DIR, 'N3M_VALID', 'n3m.valid.txt')
nf_3m_test = os.path.join(DATA_DIR, 'N3M_TEST', 'n3m.test.txt')

## Load Data

In [4]:
df_train = pd.read_csv(nf_3m_train, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_train.shape)
df_train.head()

(13675402, 3)


,CustomerID,MovieID,Rating
0,0,4884,4.0
1,0,5332,4.0
2,0,5633,4.0
3,0,6642,4.0
4,0,6922,4.0


In [5]:
df_valid = pd.read_csv(nf_3m_valid, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_valid.shape)
df_valid.head()

(1042131, 3)


,CustomerID,MovieID,Rating
0,0,1012,4.0
1,0,2977,4.0
2,0,4101,4.0
3,0,15858,4.0
4,0,16629,4.0


In [6]:
df_test = pd.read_csv(nf_3m_test, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df_test.shape)
df_test.head()

(1040428, 3)


,CustomerID,MovieID,Rating
0,0,9529,5.0
1,0,3363,4.0
2,0,5837,4.0
3,0,6017,4.0
4,0,10476,4.0


## Build Customer and Movie Maps (train)

In [7]:
customer_map = df_train.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[0:17550]
#user_map
num_users = len(customer_map)
customer_map[-1]

22203

In [8]:
movie_map = df_train.MovieID.unique()
movie_map.sort()
movie_map = movie_map[0:1000]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

1000

## Preprocess Data to create User-Movie Matrix (train)
### Uncomment to see how data is preprocessed.

In [9]:
# file = open('matrix_train.csv','w')

# def temp(i):
#     return np.where(movie_map == i)[0][0]


# for id_user in customer_map:
#     id_movie = df_train.iloc[:,1][(df_train.iloc[:,0]==id_user) & (df_train.iloc[:,1]<=1000)]
#     id_movie = id_movie.apply(temp)
#     id_rating=df_train.iloc[:,2][(df_train.iloc[:,0]==id_user) & (df_train.iloc[:,1]<=1000)]
#     ratings=np.zeros(num_movies, dtype=np.uint32)
#     ratings[id_movie-1]=id_rating
#     if sum(ratings)==0:
#         continue
#     ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
#     file.write(ratings.to_csv(index=False, header=False))
#     del id_movie
#     del id_rating
#     del ratings


# file.close()


In [10]:
X = pd.read_csv('matrix_train.csv', header=None)
print(X.shape)
X.head()

(13118, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X.shape

(13118, 1000)

## Build Customer and Movie Maps (validation)

In [12]:
customer_map = df_valid.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[:7020]
num_users = len(customer_map)
customer_map[-1]

14710

In [13]:
movie_map = df_valid.MovieID.unique()
movie_map.sort()
movie_map = movie_map[:895]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

1000

## Preprocess Data to create User-Movie Matrix (validation)
### Uncomment to see how data is preprocessed.

In [14]:
# file = open('matrix_valid.csv','w')

# def temp(i):
#     return np.where(movie_map == i)[0][0]


# for id_user in customer_map:
#     id_movie = df_valid.iloc[:,1][(df_valid.iloc[:,0]==id_user) & (df_valid.iloc[:,1]<=1000)]
#     id_movie = id_movie.apply(temp)
#     id_rating=df_valid.iloc[:,2][(df_valid.iloc[:,0]==id_user) & (df_valid.iloc[:,1]<=1000)]
#     ratings=np.zeros(1000, dtype=np.uint32)
#     ratings[id_movie-1]=id_rating
#     if sum(ratings)==0:
#         continue
#     ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
#     file.write(ratings.to_csv(index=False, header=False))
#     del id_movie
#     del id_rating
#     del ratings


# file.close()

In [15]:
X_valid = pd.read_csv('matrix_valid.csv', header=None)
print(X_valid.shape)
X_valid.head()

(2255, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_valid.shape[0]

2255

## Build Customer and Movie Maps (test)

In [17]:
customer_map = df_test.CustomerID.unique()
customer_map.sort()
customer_map = customer_map[:7020]
num_users = len(customer_map)
customer_map[-1]

14678

In [18]:
movie_map = df_test.MovieID.unique()
movie_map.sort()
movie_map = movie_map[:901]
#movie_map
num_movies = len(movie_map)
movie_map[-1]

1000

## Preprocess Data to create User-Movie Matrix (test)
### Uncomment to see how data is preprocessed.

In [19]:
# file = open('matrix_test.csv','w')

# def temp(i):
#     return np.where(movie_map == i)[0][0]


# for id_user in customer_map:
#     id_movie = df_test.iloc[:,1][(df_test.iloc[:,0]==id_user) & (df_test.iloc[:,1]<=1000)]
#     id_movie = id_movie.apply(temp)
#     id_rating=df_test.iloc[:,2][(df_test.iloc[:,0]==id_user) & (df_test.iloc[:,1]<=1000)]
#     ratings=np.zeros(1000, dtype=np.uint32)
#     ratings[id_movie-1]=id_rating
#     if sum(ratings)==0:
#         continue
#     ratings=pd.DataFrame(ratings.reshape(-1, len(ratings)))
#     file.write(ratings.to_csv(index=False, header=False))
#     del id_movie
#     del id_rating
#     del ratings


# file.close()

In [20]:
X_test = pd.read_csv('matrix_test.csv', header=None)
print(X_test.shape)
X_test.head()

(2286, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X_test.shape[0]

2286

## Create Custom Loss RMSE

In [22]:
def rmse(y_true, y_pred):
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    masked_squared_error = mask_true * K.square((y_true - y_pred))
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
    return K.sqrt(masked_mse)

## Create the Model

In [23]:
# Functional Model

encoder1 = Dense(28, activation='selu')
encoder2 = Dense(56, activation='selu')
code = Dense(56, activation='selu')
dropout = Dropout(0.65)
decoder1 = Dense(56, activation='selu')
decoder2 = Dense(28, activation='selu')
out = Dense(X.shape[1], activation='selu')


input = Input(shape=(1000,))

encoded1 = encoder1(input)
encoded2 = encoder2(encoded1)
coded = code(encoded2)
dropped = dropout(coded)
decoded1 = decoder1(dropped)
decoded2 = decoder2(decoded1)
temp_output = out(decoded2)

re_encoded1 = encoder1(temp_output)
re_encoded2 = encoder2(re_encoded1)
re_coded = code(re_encoded2)
re_dropped = dropout(re_coded)
re_decoded1 = decoder1(re_dropped)
re_decoded2 = decoder2(re_decoded1)
output = out(re_decoded2)

model = Model(inputs=input, outputs=output)
print(model.summary())
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 28)           28028       input_1[0][0]                    
                                                                 dense_6[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 56)           1624        dense_1[0][0]                    
                                                                 dense_1[1][0]                    
__________________________________________________________________________________________________
dense_3 (D

## Train the Model on the dataset

In [24]:
model.compile(loss=rmse, optimizer='sgd')

model.fit(X, X, batch_size=128, epochs=100, validation_data=(X_valid, X_valid), callbacks=[tensorboard])

Train on 13118 samples, validate on 2255 samples
Epoch 1/100
13118/13118 [==============================] - 3s 229us/step - loss: 3.3572 - val_loss: 3.4639
Epoch 2/100
13118/13118 [==============================] - 2s 184us/step - loss: 2.4903 - val_loss: 3.1486
Epoch 3/100
13118/13118 [==============================] - 2s 168us/step - loss: 1.7264 - val_loss: 2.8858
Epoch 4/100
13118/13118 [==============================] - 2s 187us/step - loss: 1.4513 - val_loss: 2.7040
Epoch 5/100
13118/13118 [==============================] - 3s 195us/step - loss: 1.3411 - val_loss: 2.5676
Epoch 6/100
13118/13118 [==============================] - 2s 156us/step - loss: 1.2617 - val_loss: 2.4616
Epoch 7/100
13118/13118 [==============================] - 2s 187us/step - loss: 1.2001 - val_loss: 2.3407
Epoch 8/100
13118/13118 [==============================] - 2s 166us/step - loss: 1.1608 - val_loss: 2.2722
Epoch 9/100
13118/13118 [==============================] - 2s 175us/step - loss: 1.1279 - val_l

13118/13118 [==============================] - 2s 163us/step - loss: 0.8772 - val_loss: 1.6056
Epoch 77/100
13118/13118 [==============================] - 2s 165us/step - loss: 0.8777 - val_loss: 1.5997
Epoch 78/100
13118/13118 [==============================] - 2s 164us/step - loss: 0.8738 - val_loss: 1.5982
Epoch 79/100
13118/13118 [==============================] - 2s 161us/step - loss: 0.8754 - val_loss: 1.5955
Epoch 80/100
13118/13118 [==============================] - 2s 159us/step - loss: 0.8750 - val_loss: 1.5987
Epoch 81/100
13118/13118 [==============================] - 2s 154us/step - loss: 0.8748 - val_loss: 1.6037
Epoch 82/100
13118/13118 [==============================] - 2s 159us/step - loss: 0.8721 - val_loss: 1.6020
Epoch 83/100
13118/13118 [==============================] - 2s 160us/step - loss: 0.8710 - val_loss: 1.5998
Epoch 84/100
13118/13118 [==============================] - 2s 161us/step - loss: 0.8699 - val_loss: 1.5869
Epoch 85/100
13118/13118 [===============

## Test the model on the test set

In [25]:
test_loss = model.evaluate(X_test, X_test)
test_loss

2286/2286 [==============================] - 0s 87us/step


1.7395454613868333